In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from scipy import stats, interpolate
from scipy.optimize import minimize
from scipy.special import gamma
from sklearn.linear_model import LinearRegression
from math import factorial
import itertools

import netCDF4 # module that reads in .nc files (built on top of HDF5 format)
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import xarray
import rioxarray

from shapely.geometry import Point, mapping
from shapely.geometry.polygon import Polygon
from pyproj import CRS, Transformer # for transforming projected coordinates to elliptical coordinates
import cartopy.crs as ccrs # for defining and transforming coordinate systems
import cartopy.feature as cfeature # to add features to a cartopy map
import cartopy.io.shapereader as shpreader

#self-libraries
from fire_utils import *

from datetime import datetime, timedelta
from cftime import num2date, date2num, DatetimeGregorian
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import matplotlib.path as mpltPath
from plot_params_mpl39 import params
import matplotlib.pylab as pylab
import seaborn as sns
pylab.rcParams.update(params)
cols_default = plt.rcParams['axes.prop_cycle'].by_key()['color']

%matplotlib inline
%config IPython.matplotlib.backend = 'retina'
%config InlineBackend.figure_format = 'retina'

## Grid scale climate-fire data

In [2]:
clim_df= init_clim_fire_grid(res= '12km', tscale= 'monthly', start_year= 1952, final_year= 2023, seas_pred_flag= True, pred_mon_ind= 3) #pred_mon_ind --> index of most recent month in calendar year
#clim_df.to_hdf('../data/clim_12km_1952_2023_data.h5', key= 'df', mode= 'w')

100%|██████████| 25/25 [01:18<00:00,  3.14s/it]


In [4]:
clim_df.dropna()

,Tmax,VPD,Prec,Solar,Wind,RH,CAPE,FFWI,Tmin,SWE_mean,...,FFWI_max3,FFWI_max7,VPD_max3,VPD_max7,Southness,Elev,Biomass,time,X,Y
32,3.960000,1.010000,103.250000,35.110001,5.63,84.820000,1.350000,8.850000,-0.22,0.812,...,15.000000,11.960000,1.400000,1.310000,-0.17,46.709999,26.336409,0,32,0
33,3.860000,0.950000,84.989998,30.879999,2.52,84.860001,1.470000,4.050000,-1.21,1.324,...,8.350000,6.220000,1.490000,1.330000,-0.06,47.369999,168.363419,0,33,0
34,3.860000,0.950000,84.989998,30.879999,2.52,84.860001,0.000000,4.050000,-1.21,1.324,...,8.350000,6.220000,1.490000,1.330000,0.04,74.360001,66.375771,0,34,0
187,4.400000,1.180000,86.279999,31.200001,4.71,82.120003,1.380000,8.420000,-1.09,0.624,...,14.540000,12.170000,1.670000,1.600000,-0.16,24.000000,34.831795,0,32,1
188,4.080000,1.170000,77.529999,31.620001,2.76,81.879997,0.000000,5.040000,-1.17,0.962,...,9.250000,7.350000,1.710000,1.550000,-0.15,34.830002,458.925995,0,33,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27565028,28.580000,18.879999,23.950001,253.190002,4.88,31.809999,71.403000,23.670000,10.02,0.000,...,47.380001,34.540001,25.120001,23.570000,0.19,699.979980,0.000000,854,138,206
27565029,28.930000,19.290001,25.250000,256.230011,4.02,30.730000,77.584000,19.959999,9.32,0.000,...,42.290001,30.459999,25.559999,23.950001,0.34,596.570007,0.000000,854,139,206
27565181,27.760000,18.020000,19.940001,255.649994,5.21,34.130001,64.431999,24.520000,11.08,0.000,...,48.869999,35.590000,24.790001,22.850000,0.05,619.330017,0.000000,854,136,207
27565182,28.590000,18.490000,21.570000,251.169998,7.68,33.490002,69.682999,36.189999,10.04,0.000,...,64.169998,49.070000,24.889999,23.190001,0.60,631.130005,0.000000,854,137,207


In [5]:
clim_df.to_hdf('../data/clim_12km_1952_2023_data.h5', key= 'df', mode= 'w')

In [27]:
clim_df_groups= clim_df.dropna().groupby(['X', 'Y'], as_index= False)
clim_df_group_keys= clim_df_groups.groups.keys()

In [28]:
len(list(clim_df_group_keys))

23903

In [24]:
clim_df_groups.get_group(list(clim_df_group_keys)[120])

,Tmax,VPD,Prec,Solar,Wind,RH,CAPE,FFWI,Tmin,SWE_mean,...,Tmax_max7,Tmin_max3,Tmin_max7,FFWI_max3,FFWI_max7,Southness,Elev,time,X,Y
18600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,120
50840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,120
83080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,120
115320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,120
147560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27422600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,850,0,120
27454840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,851,0,120
27487080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852,0,120
27519320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,853,0,120
